# Семинар по классификации текстов

В этом практическом занятии вы научитесь классифицировать тексты (твиты) на **позитивные** и **негативные** с помощью трех разных методов:

1. **TF-IDF** — простой статистический метод, который подсчитывает частоту слов
2. **Word2Vec** — метод, который учитывает смысловую близость слов
3. **BERT** — современная нейросетевая модель, понимающая контекст

Данные взяты с [Kaggle](https://www.kaggle.com/datasets/maximsuvorov/rutweetcorp?select=twitter_corpus.csv)

## Инструкция

1. Запустите все ячейки по порядку (Shift+Enter)
2. Выберите метод, который хотите протестировать
3. Сравните результаты разных методов

## Шаг 1: Установка необходимых библиотек

⚠️ **ВАЖНО:** Запустите эту ячейку один раз в начале работы. Это может занять несколько минут.

**Если возникнут ошибки с tokenizers:**
1. Попробуйте запустить в терминале: `pip install --upgrade pip wheel setuptools`
2. Или используйте только TF-IDF и Word2Vec (они работают без transformers)

In [1]:
# Установка библиотек (запускайте только один раз!)
# Обновляем pip и инструменты сборки
!pip install --upgrade pip setuptools wheel

# Основные библиотеки (обязательные для всех методов)
!pip install pandas scikit-learn gensim nltk tqdm

# PyTorch и Transformers (нужны только для BERT)
# Устанавливаем совместимые версии для Python 3.13+
# Если установка не работает, можете пропустить эти строки
# и работать только с TF-IDF и Word2Vec
!pip install torch transformers==5.1.0 accelerate==1.12.0
# Альтернатива (если не работает): используйте BERT без Trainer (см. альтернативную ячейку ниже)

print("\n✅ Установка завершена!")


✅ Установка завершена!


## Шаг 2: Загрузка и подготовка данных

Загружаем позитивные и негативные твиты из файлов.

In [50]:
df[['ttext', 'label']].to_csv('classification_data.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных
print("📂 Загрузка данных...")
df = pd.read_csv('classification_data.csv')

# Берем только текст и метку
texts = df['ttext'].astype(str).tolist()
labels = df['label'].tolist()

print(f"✅ Загружено {len(texts)} сообщений")
print(f"   Позитивных: {sum(labels)} ({sum(labels)/len(labels)*100:.1f}%)")
print(f"   Негативных: {len(labels) - sum(labels)} ({(len(labels) - sum(labels))/len(labels)*100:.1f}%)")

# Разделяем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"\n📊 Разделение данных:")
print(f"   Обучающая выборка: {len(X_train)} сообщений")
print(f"   Тестовая выборка: {len(X_test)} сообщений")

📂 Загрузка данных...
✅ Загружено 226834 сообщений
   Позитивных: 114911 (50.7%)
   Негативных: 111923 (49.3%)

📊 Разделение данных:
   Обучающая выборка: 181467 сообщений
   Тестовая выборка: 45367 сообщений


## Шаг 3: Предварительная обработка текста

Очищаем тексты от лишних символов и приводим к нижнему регистру.

In [2]:
import re

def clean_text(text):
    """Простая очистка текста"""
    # Приводим к нижнему регистру
    text = text.lower()
    # Удаляем URL
    text = re.sub(r'http\S+|www\S+', '', text)
    # Удаляем упоминания пользователей
    text = re.sub(r'@\w+', '', text)
    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("🧹 Очистка текстов...")
X_train_clean = [clean_text(text) for text in X_train]
X_test_clean = [clean_text(text) for text in X_test]

print("✅ Тексты очищены!")
print(f"\nПример исходного текста:\n  {X_train[0]}")
print(f"\nПосле очистки:\n  {X_train_clean[0]}")

🧹 Очистка текстов...
✅ Тексты очищены!

Пример исходного текста:
  @snumri ты же в центнере работаешь!! мне от кофебина до К. ехать вообще проблематично. практически никак ((

После очистки:
  ты же в центнере работаешь!! мне от кофебина до к. ехать вообще проблематично. практически никак ((


---

# 🔥 ВЫБЕРИТЕ МЕТОД ДЛЯ ОБУЧЕНИЯ 🔥

Запустите **ОДНУ** из трех следующих секций в зависимости от того, какой метод хотите попробовать:

- **Метод 1: TF-IDF** (самый быстрый, ~1 минута)
- **Метод 2: Word2Vec** (средняя скорость, ~5-10 минут)
- **Метод 3: BERT** (самый медленный, но точный, ~20-30 минут, требует transformers)

---

## 📌 Метод 1: TF-IDF + Логистическая регрессия

**Как работает:** Подсчитывает, как часто каждое слово встречается в тексте, и насколько оно уникально для данного класса.

**Плюсы:**
- Очень быстрый
- Простой для понимания
- Не требует много памяти

**Минусы:**
- Не понимает смысл слов
- Игнорирует порядок слов

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import time

print("🚀 Начинаем обучение модели TF-IDF...\n")
start_time = time.time()

# Шаг 1: Векторизация текстов
print("📝 Шаг 1/3: Преобразование текстов в числовые векторы...")
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train_clean)
X_test_tfidf = vectorizer.transform(X_test_clean)
print(f"   ✅ Создано {X_train_tfidf.shape[1]} признаков")

# Шаг 2: Обучение модели
print("\n🎓 Шаг 2/3: Обучение классификатора...")
model_tfidf = LogisticRegression(max_iter=1000, random_state=42)
model_tfidf.fit(X_train_tfidf, y_train)
print("   ✅ Модель обучена!")

# Шаг 3: Оценка качества
print("\n📊 Шаг 3/3: Оценка качества на тестовой выборке...")
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

elapsed_time = time.time() - start_time

print("\n" + "="*60)
print("📈 РЕЗУЛЬТАТЫ ДЛЯ TF-IDF")
print("="*60)
print(f"⏱️  Время обучения: {elapsed_time:.2f} секунд\n")

accuracy = accuracy_score(y_test, y_pred_tfidf)
precision = precision_score(y_test, y_pred_tfidf)
recall = recall_score(y_test, y_pred_tfidf)
f1 = f1_score(y_test, y_pred_tfidf)

print(f"🎯 Accuracy (Точность):    {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"🎯 Precision (Прецизия):   {precision:.4f} ({precision*100:.2f}%)")
print(f"🎯 Recall (Полнота):       {recall:.4f} ({recall*100:.2f}%)")
print(f"🎯 F1-Score:               {f1:.4f} ({f1*100:.2f}%)")

print("\n📋 Детальный отчет:\n")
print(classification_report(y_test, y_pred_tfidf, target_names=['Негативный', 'Позитивный']))
print("="*60)

🚀 Начинаем обучение модели TF-IDF...

📝 Шаг 1/3: Преобразование текстов в числовые векторы...
   ✅ Создано 5000 признаков

🎓 Шаг 2/3: Обучение классификатора...
   ✅ Модель обучена!

📊 Шаг 3/3: Оценка качества на тестовой выборке...

📈 РЕЗУЛЬТАТЫ ДЛЯ TF-IDF
⏱️  Время обучения: 3.45 секунд

🎯 Accuracy (Точность):    0.7295 (72.95%)
🎯 Precision (Прецизия):   0.7223 (72.23%)
🎯 Recall (Полнота):       0.7573 (75.73%)
🎯 F1-Score:               0.7394 (73.94%)

📋 Детальный отчет:

              precision    recall  f1-score   support

  Негативный       0.74      0.70      0.72     22385
  Позитивный       0.72      0.76      0.74     22982

    accuracy                           0.73     45367
   macro avg       0.73      0.73      0.73     45367
weighted avg       0.73      0.73      0.73     45367



### 🎯 ЗАДАНИЕ 1: Эксперименты с параметрами TF-IDF

Попробуйте изменить параметры и посмотрите, как это влияет на качество!

In [42]:
max_features_exp = 5000  # Попробуйте: 1000, 3000, 10000
ngram_exp = (1, 2)  # Попробуйте: (1,1), (1,2), (1,3)

print(f"🔬 Эксперимент: max_features={max_features_exp}, ngram={ngram_exp}\n")

vectorizer = TfidfVectorizer(max_features=max_features_exp, ngram_range=ngram_exp)
X_tr_exp = vectorizer.fit_transform(X_train_clean)
X_te_exp = vectorizer.transform(X_test_clean)

model_tfidf = LogisticRegression(max_iter=1000, random_state=42)
model_tfidf.fit(X_tr_exp, y_train)
y_pred_exp = model_tfidf.predict(X_te_exp)

print("📊 РЕЗУЛЬТАТЫ:")
print(f"   Accuracy:  {accuracy_score(y_test, y_pred_exp):.4f}")
print(f"   F1-Score:  {f1_score(y_test, y_pred_exp):.4f}")
print("\n💡 Запишите результаты и попробуйте другие параметры!")

🔬 Эксперимент: max_features=5000, ngram=(1, 2)

📊 РЕЗУЛЬТАТЫ:
   Accuracy:  0.7295
   F1-Score:  0.7394

💡 Запишите результаты и попробуйте другие параметры!


### 🔍 Посмотрим, какие слова модель считает самыми важными!

In [5]:
# Топ-10 слов для каждого класса
import numpy as np

features = vectorizer.get_feature_names_out()
coefs = model_tfidf.coef_[0]

top_pos = np.argsort(coefs)[-10:]
top_neg = np.argsort(coefs)[:10]

print("😊 Топ-10 ПОЗИТИВНЫХ слов:\n")
for idx in reversed(top_pos):
    print(f"   {features[idx]:20s} ({coefs[idx]:.3f})")

print("\n😞 Топ-10 НЕГАТИВНЫХ слов:\n")
for idx in top_neg:
    print(f"   {features[idx]:20s} ({coefs[idx]:.3f})")

print("\n❓ Вопросы:")
print("   - Согласны ли вы с этими словами?")
print("   - Какие слова вас удивили?")

😊 Топ-10 ПОЗИТИВНЫХ слов:

   dd                   (9.947)
   ddd                  (7.749)
   dddd                 (4.757)
   приятно              (4.428)
   xd                   (4.121)
   ахаха                (3.912)
   ахахах               (3.872)
   ахах                 (3.825)
   ржу                  (3.815)
   спасибо              (3.685)

😞 Топ-10 НЕГАТИВНЫХ слов:

   o_o                  (-9.059)
   о_о                  (-7.174)
   обидно               (-6.924)
   cio_optimal          (-6.789)
   to_over_kill         (-6.303)
   печально             (-6.032)
   грустно              (-5.769)
   do_or_die_xxx        (-5.166)
   печаль               (-4.992)
   99                   (-4.780)

❓ Вопросы:
   - Согласны ли вы с этими словами?
   - Какие слова вас удивили?


## 📌 Метод 2: Word2Vec + Логистическая регрессия

**Как работает:** Создает векторные представления слов, где похожие по смыслу слова находятся близко друг к другу.

**Плюсы:**
- Понимает смысловую близость слов
- Хорошо работает с синонимами

**Минусы:**
- Требует больше времени на обучение
- Усредняет значения слов в предложении

In [6]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
import numpy as np
import time

print("🚀 Начинаем обучение модели Word2Vec...\n")
start_time = time.time()

# Шаг 1: Токенизация
print("📝 Шаг 1/4: Разбиение текстов на слова...")
X_train_tokens = [text.split() for text in X_train_clean]
X_test_tokens = [text.split() for text in X_test_clean]
print(f"   ✅ Обработано {len(X_train_tokens)} текстов")

# Шаг 2: Обучение Word2Vec
print("\n🎓 Шаг 2/4: Обучение модели Word2Vec (может занять несколько минут)...")
w2v_model = Word2Vec(
    sentences=X_train_tokens,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    epochs=10
)
print("   ✅ Word2Vec модель обучена!")
print(f"   📊 Словарь содержит {len(w2v_model.wv)} слов")

# Шаг 3: Преобразование текстов в векторы
print("\n🔄 Шаг 3/4: Преобразование текстов в векторы...")

def text_to_vector(tokens, model):
    """Усредняем векторы всех слов в тексте"""
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

X_train_w2v = np.array([text_to_vector(tokens, w2v_model) for tokens in X_train_tokens])
X_test_w2v = np.array([text_to_vector(tokens, w2v_model) for tokens in X_test_tokens])
print(f"   ✅ Создано {X_train_w2v.shape[1]} признаков для каждого текста")

# Шаг 4: Обучение классификатора
print("\n🎓 Шаг 4/4: Обучение классификатора...")
model_w2v = LogisticRegression(max_iter=1000, random_state=42)
model_w2v.fit(X_train_w2v, y_train)
print("   ✅ Классификатор обучен!")

# Оценка качества
print("\n📊 Оценка качества на тестовой выборке...")
y_pred_w2v = model_w2v.predict(X_test_w2v)

elapsed_time = time.time() - start_time

print("\n" + "="*60)
print("📈 РЕЗУЛЬТАТЫ ДЛЯ WORD2VEC")
print("="*60)
print(f"⏱️  Время обучения: {elapsed_time:.2f} секунд\n")

accuracy = accuracy_score(y_test, y_pred_w2v)
precision = precision_score(y_test, y_pred_w2v)
recall = recall_score(y_test, y_pred_w2v)
f1 = f1_score(y_test, y_pred_w2v)

print(f"🎯 Accuracy (Точность):    {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"🎯 Precision (Прецизия):   {precision:.4f} ({precision*100:.2f}%)")
print(f"🎯 Recall (Полнота):       {recall:.4f} ({recall*100:.2f}%)")
print(f"🎯 F1-Score:               {f1:.4f} ({f1*100:.2f}%)")

print("\n📋 Детальный отчет:\n")
print(classification_report(y_test, y_pred_w2v, target_names=['Негативный', 'Позитивный']))
print("="*60)

🚀 Начинаем обучение модели Word2Vec...

📝 Шаг 1/4: Разбиение текстов на слова...
   ✅ Обработано 181467 текстов

🎓 Шаг 2/4: Обучение модели Word2Vec (может занять несколько минут)...


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


   ✅ Word2Vec модель обучена!
   📊 Словарь содержит 93672 слов

🔄 Шаг 3/4: Преобразование текстов в векторы...
   ✅ Создано 100 признаков для каждого текста

🎓 Шаг 4/4: Обучение классификатора...
   ✅ Классификатор обучен!

📊 Оценка качества на тестовой выборке...

📈 РЕЗУЛЬТАТЫ ДЛЯ WORD2VEC
⏱️  Время обучения: 8.26 секунд

🎯 Accuracy (Точность):    0.7669 (76.69%)
🎯 Precision (Прецизия):   0.7624 (76.24%)
🎯 Recall (Полнота):       0.7842 (78.42%)
🎯 F1-Score:               0.7731 (77.31%)

📋 Детальный отчет:

              precision    recall  f1-score   support

  Негативный       0.77      0.75      0.76     22385
  Позитивный       0.76      0.78      0.77     22982

    accuracy                           0.77     45367
   macro avg       0.77      0.77      0.77     45367
weighted avg       0.77      0.77      0.77     45367



### 🎯 ЗАДАНИЕ 2: Исследуем векторные представления

Word2Vec создает векторы слов. Давайте найдем похожие слова!

In [7]:
# Найдем похожие слова
test_words = ['хорошо', 'плохо', 'люблю', 'счастье']

print("🔍 Поиск похожих слов:\n")
for word in test_words:
    if word in w2v_model.wv:
        similar = w2v_model.wv.most_similar(word, topn=5)
        print(f"📌 '{word}':")
        for sim_word, score in similar:
            print(f"   {sim_word:15s} ({score:.3f})")
        print()
    else:
        print(f"⚠️  '{word}' не найдено\n")

print("💡 TODO: Замените слова на свои и запустите снова!")
print("\n❓ Вопросы:")
print("   - Согласны ли вы с похожими словами?")
print("   - Какие пары удивили?")

🔍 Поиск похожих слов:

📌 'хорошо':
   хорошо,         (0.750)
   весело          (0.680)
   круто           (0.676)
   классно         (0.673)
   хорошо.         (0.657)

📌 'плохо':
   плохо,          (0.796)
   грустно         (0.786)
   больно          (0.753)
   хреново         (0.744)
   плохо.          (0.735)

📌 'люблю':
   обожаю          (0.781)
   люблю,          (0.702)
   любила          (0.684)
   люблю!          (0.683)
   обнимаю         (0.643)

📌 'счастье':
   чудо            (0.769)
   ахуенное        (0.760)
   смешное         (0.751)
   классное        (0.751)
   красивое        (0.746)

💡 TODO: Замените слова на свои и запустите снова!

❓ Вопросы:
   - Согласны ли вы с похожими словами?
   - Какие пары удивили?


## 📌 Метод 3: BERT (Transformer)

**Как работает:** Использует глубокую нейронную сеть, которая понимает контекст слов и их взаимосвязи.

**Плюсы:**
- Самое глубокое понимание текста
- Учитывает контекст и порядок слов
- Современный state-of-the-art метод

**Минусы:**
- Самый медленный метод
- Требует много памяти
- Требует установки transformers

**⚠️ ВНИМАНИЕ:** 
- Обучение может занять 20-30 минут!
- Если transformers не установлен, используйте TF-IDF или Word2Vec

In [ ]:
# Проверяем, установлен ли transformers
try:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
    import torch
    from torch.utils.data import Dataset
    print("✅ Библиотека transformers установлена, можно продолжать!\n")
except ImportError:
    print("❌ ОШИБКА: Библиотека transformers не установлена!")
    print("\nПопробуйте установить:")
    print("  !pip install --upgrade pip wheel setuptools")
    print("  !pip install torch transformers")
    print("\nИли используйте TF-IDF и Word2Vec вместо BERT.")
    raise

✅ Библиотека transformers установлена, можно продолжать!



In [43]:
import time

print("🚀 Начинаем обучение модели BERT...\n")
print("⚠️  Это займет значительное время! Наберитесь терпения...\n")
start_time = time.time()

# Используем облегченную русскоязычную модель
model_name = "cointegrated/rubert-tiny2"

# Шаг 1: Загрузка токенизатора и модели
print("📝 Шаг 1/4: Загрузка предобученной модели BERT...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_bert = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2
)
print("   ✅ Модель загружена!")

for name, param in model_bert.named_parameters():
     if name.startswith("cls") or name.startswith("bert"): 
        param.requires_grad = False

# Шаг 2: Подготовка данных
print("\n🔄 Шаг 2/4: Подготовка данных для обучения...")

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Для ускорения используем подвыборку данных (можете увеличить для лучшего качества)
sample_size = min(200000, len(X_train_clean))  
X_train_sample = X_train_clean[:sample_size]
y_train_sample = y_train[:sample_size]

train_dataset = TextDataset(X_train_sample, y_train_sample, tokenizer)
test_dataset = TextDataset(X_test_clean, y_test, tokenizer)
print(f"   ✅ Подготовлено {len(train_dataset)} обучающих примеров")

# Шаг 3: Настройка обучения
print("\n⚙️  Шаг 3/4: Настройка параметров обучения...")
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Всего одна эпоха для скорости
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",  # Исправлено: eval_strategy вместо evaluation_strategy
    save_strategy="no",
)

trainer = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
print("   ✅ Настройки готовы!")

# Шаг 4: Обучение
print("\n🎓 Шаг 4/4: Обучение модели (это займет время, следите за прогрессом)...\n")
trainer.train()
print("\n   ✅ Обучение завершено!")

# Оценка качества
print("\n📊 Оценка качества на тестовой выборке...")
predictions = trainer.predict(test_dataset)
y_pred_bert = np.argmax(predictions.predictions, axis=1)

elapsed_time = time.time() - start_time

print("\n" + "="*60)
print("📈 РЕЗУЛЬТАТЫ ДЛЯ BERT")
print("="*60)
print(f"⏱️  Время обучения: {elapsed_time:.2f} секунд ({elapsed_time/60:.1f} минут)\n")

accuracy = accuracy_score(y_test, y_pred_bert)
precision = precision_score(y_test, y_pred_bert)
recall = recall_score(y_test, y_pred_bert)
f1 = f1_score(y_test, y_pred_bert)

print(f"🎯 Accuracy (Точность):    {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"🎯 Precision (Прецизия):   {precision:.4f} ({precision*100:.2f}%)")
print(f"🎯 Recall (Полнота):       {recall:.4f} ({recall*100:.2f}%)")
print(f"🎯 F1-Score:               {f1:.4f} ({f1*100:.2f}%)")

print("\n📋 Детальный отчет:\n")
print(classification_report(y_test, y_pred_bert, target_names=['Негативный', 'Позитивный']))
print("="*60)

🚀 Начинаем обучение модели BERT...

⚠️  Это займет значительное время! Наберитесь терпения...

📝 Шаг 1/4: Загрузка предобученной модели BERT...


Loading weights: 100%|██████████| 55/55 [00:00<00:00, 1856.72it/s, Materializing param=bert.pooler.dense.weight]                              
BertForSequenceClassification LOAD REPORT from: cointegrated/rubert-tiny2
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/archi

   ✅ Модель загружена!

🔄 Шаг 2/4: Подготовка данных для обучения...


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


   ✅ Подготовлено 181467 обучающих примеров

⚙️  Шаг 3/4: Настройка параметров обучения...
   ✅ Настройки готовы!

🎓 Шаг 4/4: Обучение модели (это займет время, следите за прогрессом)...



Epoch,Training Loss,Validation Loss
1,0.553241,0.541981



   ✅ Обучение завершено!

📊 Оценка качества на тестовой выборке...



📈 РЕЗУЛЬТАТЫ ДЛЯ BERT
⏱️  Время обучения: 174.91 секунд (2.9 минут)

🎯 Accuracy (Точность):    0.7649 (76.49%)
🎯 Precision (Прецизия):   0.7826 (78.26%)
🎯 Recall (Полнота):       0.7421 (74.21%)
🎯 F1-Score:               0.7618 (76.18%)

📋 Детальный отчет:

              precision    recall  f1-score   support

  Негативный       0.75      0.79      0.77     22385
  Позитивный       0.78      0.74      0.76     22982

    accuracy                           0.76     45367
   macro avg       0.77      0.77      0.76     45367
weighted avg       0.77      0.76      0.76     45367



---

### 🎉 Проверка модели BERT на своих примерах

После обучения модели вы можете протестировать её на собственных текстах!

In [44]:
# Тестовые примеры (можете изменить на свои!)
test_examples = [
    "Какой прекрасный день! Настроение отличное :)",
    "Ужасная погода, все плохо :(",
    "Опять все сломалось, ничего не работает...",
    "Какой прекрасный день! Настроение отличное",
]

# Функция для предсказания (выберите нужную модель!)
def predict_sentiment(texts):
    """
    Предсказание тональности текстов
    """
    cleaned = [clean_text(text) for text in texts]
    
    if torch.backends.mps.is_built():
        device = torch.device("mps") 
    encodings = tokenizer(cleaned, truncation=True, padding=True, max_length=128, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model_bert(**encodings)
        predictions = torch.argmax(outputs.logits, dim=1).to('cpu').numpy()
    
    return predictions

print(f"🔮 Тестирование модели BERT:\n")
predictions = predict_sentiment(test_examples)

for text, pred in zip(test_examples, predictions):
    sentiment = "😊 Позитивный" if pred == 1 else "😞 Негативный"
    print(f"Текст: {text}")
    print(f"Предсказание: {sentiment}\n")

🔮 Тестирование модели BERT:

Текст: Какой прекрасный день! Настроение отличное :)
Предсказание: 😊 Позитивный

Текст: Ужасная погода, все плохо :(
Предсказание: 😞 Негативный

Текст: Опять все сломалось, ничего не работает...
Предсказание: 😞 Негативный

Текст: Какой прекрасный день! Настроение отличное
Предсказание: 😊 Позитивный



### 🧪 ЗАДАНИЕ 3: Протестируйте модели на своих примерах

Придумайте свои твиты и проверьте все три модели!

In [48]:
my_tweets = [
    "Сегодня отличный день!",  # Позитивный
    "Все ужасно :(",  # Негативный
    "Ну наконец-то...",  # Неоднозначный
    # TODO: Добавьте еще 5-7 своих примеров
]

# TODO: Не забудте добавить разметку для своих примеров
my_expected = [1, 0, 1]  # 1 = позитивный, 0 = негативный

print("🧪 ТЕСТИРОВАНИЕ\n" + "="*60)

for i, tweet in enumerate(my_tweets):
    print(f"\n📝 {i+1}. {tweet}")
    print(f"   💭 Ожидаю: {'😊 Позитивный' if my_expected[i]==1 else '😞 Негативный'}")
    
    cleaned = clean_text(tweet)
    
    # TF-IDF
    try:
        p = model_tfidf.predict(vectorizer.transform([cleaned]))[0]
        match = '✅' if p == my_expected[i] else '❌'
        print(f"   📊 TF-IDF:   {'😊' if p==1 else '😞'} {match}")
    except:
        print("   📊 TF-IDF:   не обучена")
    
    # Word2Vec
    try:
        v = text_to_vector(cleaned.split(), w2v_model).reshape(1,-1)
        p = model_w2v.predict(v)[0]
        match = '✅' if p == my_expected[i] else '❌'
        print(f"   🌐 Word2Vec: {'😊' if p==1 else '😞'} {match}")
    except:
        print("   🌐 Word2Vec: не обучена")
    
    # BERT
    try:    
        if torch.backends.mps.is_built():
            device = torch.device("mps") 

        encodings = tokenizer([cleaned], truncation=True, padding=True, max_length=128, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model_bert(**encodings)
            p = torch.argmax(outputs.logits, dim=1).to('cpu').numpy()
        match = '✅' if p == my_expected[i] else '❌'
        print(f"   🤖 BERT:     {'😊' if p==1 else '😞'} {match}")
    except:
        print("   🤖 BERT:     не обучена")

print("\n" + "="*60)
print("\n❓ Вопросы:")
print("   - Все ли модели согласны?")
print("   - На каких примерах они расходятся?")
print("   - Какая модель точнее?")

🧪 ТЕСТИРОВАНИЕ

📝 1. Сегодня отличный день!
   💭 Ожидаю: 😊 Позитивный
   📊 TF-IDF:   😊 ✅
   🌐 Word2Vec: 😊 ✅
   🤖 BERT:     😊 ✅

📝 2. Все ужасно :(
   💭 Ожидаю: 😞 Негативный
   📊 TF-IDF:   😞 ✅
   🌐 Word2Vec: 😞 ✅
   🤖 BERT:     😞 ✅

📝 3. Ну наконец-то...
   💭 Ожидаю: 😊 Позитивный
   📊 TF-IDF:   😊 ✅
   🌐 Word2Vec: 😞 ❌
   🤖 BERT:     😊 ✅


❓ Вопросы:
   - Все ли модели согласны?
   - На каких примерах они расходятся?
   - Какая модель точнее?


### 📊 ЗАДАНИЕ 4: Итоговая таблица сравнения

Заполните таблицу с результатами всех методов:

| Метод | Accuracy | F1-Score | Время | Плюсы | Минусы |
|-------|----------|----------|-------|-------|--------|
| TF-IDF | ? | ? | ? | ? | ? |
| Word2Vec | ? | ? | ? | ? | ? |
| BERT | ? | ? | ? | ? | ? |

---

---

## 📊 Объяснение метрик

**Accuracy (Точность)** — доля правильных предсказаний среди всех предсказаний
- Формула: (Правильные предсказания) / (Все предсказания)
- Пример: Если из 100 твитов модель правильно классифицировала 85, то accuracy = 0.85 (85%)

**Precision (Прецизия)** — из всех текстов, которые модель назвала позитивными, какая доля действительно позитивная
- Формула: (Правильно найденные позитивные) / (Все предсказанные как позитивные)
- Важна, когда нужно избежать ложных тревог

**Recall (Полнота)** — из всех действительно позитивных текстов, какую долю модель нашла
- Формула: (Правильно найденные позитивные) / (Все реально позитивные)
- Важна, когда нужно найти все позитивные примеры

**F1-Score** — среднее гармоническое между Precision и Recall
- Показывает баланс между точностью и полнотой
- Используется для общей оценки качества

---

## ✅ Задание

1. Обучите все три модели (TF-IDF, Word2Vec, BERT) или хотя бы две, если BERT не установился
2. Запишите результаты в таблицу:

| Метод | Accuracy | Precision | Recall | F1-Score | Время обучения |
|-------|----------|-----------|--------|----------|----------------|
| TF-IDF | ? | ? | ? | ? | ? |
| Word2Vec | ? | ? | ? | ? | ? |
| BERT | ? | ? | ? | ? | ? |

3. Ответьте на вопросы:
   - Какая модель показала лучший результат?
   - Какая модель обучалась быстрее всего?
   - Какой метод вы бы выбрали для реального проекта и почему?
   - Попробуйте добавить свои примеры текстов в последнюю ячейку. Все ли модели правильно их классифицируют?

---

## 💡 Советы по устранению проблем

**Если не устанавливается transformers или accelerate:**
```bash
# В терминале:
pip install --upgrade pip wheel setuptools
pip install torch --index-url https://download.pytorch.org/whl/cpu
pip install transformers>=4.45.0 accelerate>=0.35.0
```

**Для Python 3.13+:** Обязательно используйте версии `transformers>=4.45.0` и `accelerate>=0.35.0`

**Если возникает ошибка 'keep_torch_compile' при обучении BERT:**

Это проблема несовместимости версий. У вас есть ДВА решения:

1. **Обновить библиотеки (рекомендуется):**
```bash
pip install --upgrade transformers accelerate
```

2. **Использовать альтернативную ячейку с BERT** (без Trainer API):
   - В notebook есть две ячейки для обучения BERT
   - Если основная не работает, используйте альтернативную (упрощенный метод)
   - Она не требует accelerate и работает стабильнее

**Если мало памяти для BERT:**
- Уменьшите `sample_size` (например, до 5000)
- Уменьшите `per_device_train_batch_size` / `batch_size` (например, до 8)
- Уменьшите `num_train_epochs` до 1

**Если BERT вообще не работает:**
- Это нормально! TF-IDF и Word2Vec дают хорошие результаты (72-75% accuracy)
- Используйте их для сравнения
- BERT требует больше ресурсов и более сложную настройку
- Для учебных целей достаточно двух методов